In [1]:
# |default_exp dataset

In [2]:
#| export
import pandas as pd
import numpy as np
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
from torch_geometric.data import Data, DataLoader
import torch
import seaborn as sbn
import torch.nn.functional as F
from tqdm import tqdm

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
#| export
def good_luck():
    return True

In [4]:
# |export 
class LenMatchBatchSampler(torch.utils.data.BatchSampler):
    def __iter__(self):
        buckets = [[]] * 100
        yielded = 0

        for idx in self.sampler:
            s = self.sampler.data_source[idx]
            if isinstance(s,tuple): L = s[0]["mask"].sum()
            else: L = s["mask"].sum()
            L = max(1,L // 16) 
            if len(buckets[L]) == 0:  buckets[L] = []
            buckets[L].append(idx)
            
            if len(buckets[L]) == self.batch_size:
                batch = list(buckets[L])
                yield batch
                yielded += 1
                buckets[L] = []
                
        batch = []
        leftover = [idx for bucket in buckets for idx in bucket]

        for idx in leftover:
            batch.append(idx)
            if len(batch) == self.batch_size:
                yielded += 1
                yield batch
                batch = []

        if len(batch) > 0 and not self.drop_last:
            yielded += 1
            yield batch
            
def dict_to(x, device='cuda'):
    return {k:x[k].to(device) for k in x}

def to_device(x, device='cuda'):
    return tuple(dict_to(e,device) for e in x)

class DeviceDataLoader:
    def __init__(self, dataloader, device='cuda'):
        self.dataloader = dataloader
        self.device = device
    
    def __len__(self):
        return len(self.dataloader)
    
    def __iter__(self):
        for batch in self.dataloader:
            yield tuple(dict_to(x, self.device) for x in batch)

In [5]:
# |export 
def encode_rna_sequence(seq):
    L = len(seq)

    # Initialize the tensor with zeros
    tensor = np.zeros((L, L, 8))

    # Define valid base pairs
    valid_pairs = [
        ("A", "U"),
        ("U", "A"),
        ("U", "G"),
        ("G", "U"),
        ("G", "C"),
        ("C", "G"),
    ]

    for i in range(L):
        for j in range(L):
            # Check for valid base pairs
            if (seq[i], seq[j]) in valid_pairs:
                channel = valid_pairs.index((seq[i], seq[j]))
                tensor[i, j, channel] = 1
            # Check for diagonal
            elif i == j:
                tensor[i, j, 6] = 1
            # If not a valid pair and not on the diagonal, set the last channel
            else:
                tensor[i, j, 7] = 1

    return tensor


def generate_adj_matrix(file_path, max_len, prob=0.5):
    """
    Reads a TXT file of base pair probabilities and generates an n x n matrix.

    Args:
    - file_path (str): Path to the TXT file.

    Returns:
    - np.array: An n x n matrix of base pair probabilities.
    """
    # Read the data using pandas
    data = pd.read_csv(file_path, sep=" ", header=None, names=["pos1", "pos2", "prob"])
    data = data.query('prob>@prob')

    # Initialize the matrix
    matrix = np.zeros((max_len, max_len))

    # Populate the matrix
    for _, row in data.iterrows():
        pos1, pos2, prob = int(row["pos1"]), int(row["pos2"]), row["prob"]
        matrix[pos1 - 1, pos2 - 1] = prob
        matrix[pos2 - 1, pos1 - 1] = prob

    return torch.tensor(matrix, dtype=torch.int)

def generate_edge_data(file_path):
    # Read the file into a DataFrame
    data = pd.read_csv(file_path, sep=" ", header=None, names=["pos1", "pos2", "prob"])
    
    # Convert the pos1 and pos2 columns to 0-based indices and then to a tensor for edge index
    edge_index = torch.tensor([data["pos1"].values - 1, data["pos2"].values - 1], dtype=torch.long)
    
    # Convert the prob column to a tensor for edge features
    edge_features = torch.tensor(data["prob"].values, dtype=torch.float).unsqueeze(1)  # Adding an extra dimension
    
    return edge_index, edge_features



class RNA_DatasetV0(Dataset):
    def __init__(self, df, mask_only=False,prob_for_adj = 0.5, **kwargs):

        self.seq_map = {'A':0,'C':1,'G':2,'U':3}
        self.Lmax = 206
        self.prob_for_adj = prob_for_adj
        df['L'] = df.sequence.apply(len)
        df_2A3 = df.loc[df.experiment_type=='2A3_MaP'].reset_index(drop=True)
        df_DMS = df.loc[df.experiment_type=='DMS_MaP'].reset_index(drop=True)


        self.seq = df_2A3['sequence'].values
        self.L = df_2A3['L'].values
        
        self.react_2A3 = df_2A3[[c for c in df_2A3.columns if \
                                 'reactivity_0' in c]].values
        self.react_DMS = df_DMS[[c for c in df_DMS.columns if \
                                 'reactivity_0' in c]].values
        self.react_err_2A3 = df_2A3[[c for c in df_2A3.columns if \
                                 'reactivity_error_0' in c]].values
        self.react_err_DMS = df_DMS[[c for c in df_DMS.columns if \
                                'reactivity_error_0' in c]].values
        self.sn_2A3 = df_2A3['signal_to_noise'].values
        self.sn_DMS = df_DMS['signal_to_noise'].values
        self.bpp = df_2A3['bpp'].values
        self.mask_only = mask_only
        
    def __len__(self):
        return len(self.seq)  
    
    def __getitem__(self, idx):
        seq = self.seq[idx]
        if self.mask_only:
            mask = torch.zeros(self.Lmax, dtype=torch.bool)
            mask[:len(seq)] = True
            return {'mask':mask},{'mask':mask}
        adj_matrix = generate_adj_matrix(self.bpp[idx], self.Lmax, self.prob_for_adj)
        seq = [self.seq_map[s] for s in seq]
        seq = np.array(seq)
        mask = torch.zeros(self.Lmax, dtype=torch.bool)
        mask[:len(seq)] = True
        seq = np.pad(seq,(0,self.Lmax-len(seq)))
        
        react = torch.from_numpy(np.stack([self.react_2A3[idx],
                                           self.react_DMS[idx]],-1))
        react_err = torch.from_numpy(np.stack([self.react_err_2A3[idx],
                                               self.react_err_DMS[idx]],-1))
        return {"seq": torch.from_numpy(seq), "mask": mask, "adj_matrix": adj_matrix}, {
            "react": react,
            "react_err": react_err,
            "mask": mask,
        }
        
        



class RNA_DatasetV0G(Dataset):
    def __init__(self, df, path_to_bpp_folder, mask_only=False, **kwargs):

        self.seq_map = {'A':0,'C':1,'G':2,'U':3}
        self.Lmax = 206
        df['L'] = df.sequence.apply(len)
        df_2A3 = df.loc[df.experiment_type=='2A3_MaP'].reset_index(drop=True)
        df_DMS = df.loc[df.experiment_type=='DMS_MaP'].reset_index(drop=True)


        self.seq = df_2A3['sequence'].values
        self.L = df_2A3['L'].values
        
        self.react_2A3 = df_2A3[[c for c in df_2A3.columns if \
                                 'reactivity_0' in c]].values
        self.react_DMS = df_DMS[[c for c in df_DMS.columns if \
                                 'reactivity_0' in c]].values
        self.react_err_2A3 = df_2A3[[c for c in df_2A3.columns if \
                                 'reactivity_error_0' in c]].values
        self.react_err_DMS = df_DMS[[c for c in df_DMS.columns if \
                                'reactivity_error_0' in c]].values
        self.sn_2A3 = df_2A3['signal_to_noise'].values
        self.sn_DMS = df_DMS['signal_to_noise'].values
        self.bpp = df_2A3['bpp'].values
        self.mask_only = mask_only
        
    def __len__(self):
        return len(self.seq)  
    
    def __getitem__(self, idx):
        seq = self.seq[idx]
        if self.mask_only:
            mask = torch.zeros(self.Lmax, dtype=torch.bool)
            mask[:len(seq)] = True
            return {'mask':mask},{'mask':mask}
        edge_index, edge_features = generate_edge_data(self.bpp[idx])
        seq = [self.seq_map[s] for s in seq]
        seq = np.array(seq)

        
        react = torch.from_numpy(np.stack([self.react_2A3[idx],
                                           self.react_DMS[idx]],-1))
        react_err = torch.from_numpy(np.stack([self.react_err_2A3[idx],
                                               self.react_err_DMS[idx]],-1))
        return Data(x=torch.from_numpy(seq), edge_index=edge_index, edge_features= edge_features, y=react, y_err=react_err)
    
    

class LenMatchBatchSampler(torch.utils.data.BatchSampler):
    def __iter__(self):
        buckets = [[]] * 100
        yielded = 0

        for idx in self.sampler:
            s = self.sampler.data_source[idx]
            if isinstance(s, tuple):
                L = s[0]["mask"].sum()
            else:
                L = s["mask"].sum()
            L = max(1, L // 16)
            if len(buckets[L]) == 0:
                buckets[L] = []
            buckets[L].append(idx)

            if len(buckets[L]) == self.batch_size:
                batch = list(buckets[L])
                yield batch
                yielded += 1
                buckets[L] = []

        batch = []
        leftover = [idx for bucket in buckets for idx in bucket]

        for idx in leftover:
            batch.append(idx)
            if len(batch) == self.batch_size:
                yielded += 1
                yield batch
                batch = []

        if len(batch) > 0 and not self.drop_last:
            yielded += 1
            yield batch

In [6]:
class CFG:
    path = Path("../data/split")
    pathbb = Path("../data/Ribonanza_bpp_files")
    split_id = 'v0'
    bs = 16
    num_workers = 8
    device = 'cpu'
    adjnact_prob = 0.5



fns = list(CFG.pathbb.rglob("*.txt"))
bpp_df = pd.DataFrame({"bpp": fns})
bpp_df['sequence_id'] = bpp_df['bpp'].apply(lambda x: x.stem)
df_train = pd.read_parquet(CFG.path / f"train_data_{CFG.split_id}.parquet")
df_valid = pd.read_parquet(CFG.path / f"valid_data_{CFG.split_id}.parquet")
df_train = df_train.merge(bpp_df, on="sequence_id", how="left").reset_index(drop=True)
df_valid = df_valid.merge(bpp_df, on="sequence_id", how="left").reset_index(drop=True)
ds_train = RNA_DatasetV0(df_train, path_to_bpp_folder=CFG.pathbb)
ds_train_len = RNA_DatasetV0(df_train, path_to_bpp_folder=CFG.pathbb, mask_only=True)
sampler_train = torch.utils.data.RandomSampler(ds_train_len)
len_sampler_train = LenMatchBatchSampler(sampler_train, batch_size=CFG.bs,
            drop_last=True)
dl_train = DeviceDataLoader(torch.utils.data.DataLoader(ds_train, 
            batch_sampler=len_sampler_train, num_workers=CFG.num_workers,
            persistent_workers=True), CFG.device)

In [7]:
#|hide
#|eval: false
from nbdev.doclinks import nbdev_export
nbdev_export()